## **Structured Data Extraction from Resume Files**

**Installations**

In [1]:
!pip install python-docx
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 62.5 MB/s eta 0:00:00


In [2]:
!pip install pip==23.2.1

!apt-get install -y antiword unrtf poppler-utils tesseract-ocr \
    flac ffmpeg lame libmad0 libsox-fmt-mp3 sox

!pip install textract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libid3tag0 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 libwavpack1
Suggested packages:
  lame-doc libsox-fmt-all
The following NEW packages will be installed:
  antiword flac lame libid3tag0 libmad0 libopencore-amrnb0 libopencore-amrwb0
  libsox-fmt-alsa libsox-fmt-base libsox-fmt-mp3 libsox3 libwavpack1
  poppler-utils sox unrtf
0 upgraded, 15 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,259 kB of archives.
After this operation, 4,091 kB of

**Imports**

In [2]:
import os
import pandas as pd
from docx import Document
import fitz
import textract

Upload a 'resumes' folder containing subfolders — Peoplesoft, React Developer, SQL Developer, and Workday — so that resumes can be extracted and categorized based on their respective subfolder names.



In [14]:
# Root folder containing subfolders for each role
resume_root_folder = "./resumes"

# Function to extract text from .doc files
def extract_text_from_doc(file_path):
    try:
        return textract.process(file_path).decode("utf-8")
    except Exception as e:
        print(f"Could not read .doc file {file_path}: {e}")
        return None

# Function to extract text from .docx files
def extract_text_from_docx(file_path):
    try:
        doc = Document(file_path)
        return ' '.join([para.text for para in doc.paragraphs])
    except Exception as e:
        print(f"Could not read .docx file {file_path}: {e}")
        return None

# Function to extract text from PDF files
def extract_text_from_pdf(file_path):
    try:
        doc = fitz.open(file_path)
        return ' '.join([page.get_text() for page in doc]).strip()
    except Exception as e:
        print(f"Could not read PDF {file_path}: {e}")
        return None

# Resume-related keywords for validation
def is_resume_content(text):
    if not text or len(text.split()) < 100:
        return False

    text_lower = text.lower()

    # Flexible section keywords — roots and common variations
    section_keywords = [
        "experience", "education", "skill", "project", "summary",
        "certification", "objective", "responsibilit", "training",
        "internship", "career objective", "technical", "professional", "achievement"
    ]
    # Count how many keywords appear (even partial matches like "skillset", "responsibilities")
    match_count = sum(keyword in text_lower for keyword in section_keywords)

    return match_count >= 2

# Go through each role folder and process resumes
results = []

for role_folder in os.listdir(resume_root_folder):
    full_path = os.path.join(resume_root_folder, role_folder)
    if not os.path.isdir(full_path):
        continue  # Skip if not a folder

    for filename in os.listdir(full_path):
        file_path = os.path.join(full_path, filename)
        ext = filename.lower().split('.')[-1]
        text = None

        if ext == "docx":
            text = extract_text_from_docx(file_path)
        elif ext == "pdf":
            text = extract_text_from_pdf(file_path)
        elif ext == "doc":
            text = extract_text_from_doc(file_path)
        else:
            text = None  # Unsupported format

        # Validate resume content
        if not text or not is_resume_content(text):
            text = "Invalid Resume Content"

        results.append({
            "filename": filename,
            "content": text.strip(),
            "ext": ext,
            "role": role_folder
        })

# Save results to CSV
df = pd.DataFrame(results)
df.to_csv("resume_dataset_labeled.csv", index=False)

print("CSV 'resume_dataset_labeled.csv' created with role labels and content checks.")

CSV 'resume_dataset_labeled.csv' created with role labels and content checks.


In [15]:
df=pd.read_csv("resume_dataset_labeled.csv")
df.sample(5)

,filename,content,ext,role
63,React Dev_Krishna Kanth_Musquare Technologies....,Ui-Developer/ React JS Developer NAME: KRISHN...,docx,React Developer
18,Harikrishna Akula_Hexaware.doc,Harikrishna Akula\n\n\n\n Summary:\n ❖ 5.2 ...,doc,Workday Consultant
77,kamballapradeep.docx,KAMBALLA PRADEEP ...,docx,SQL Developer
64,Reactjs Developer_Prabakaran_Musquare Technolo...,Page | 1 \n \nName: M. Prabakaran \nTitle: UI...,pdf,React Developer
53,Internship_Ravali_Musquare Technologies (1).docx,Name: Ravali P ...,docx,React Developer


In [16]:
df.shape

(80, 4)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  80 non-null     object
 1   content   80 non-null     object
 2   ext       80 non-null     object
 3   role      80 non-null     object
dtypes: object(4)
memory usage: 2.6+ KB


In [18]:
df["role"].value_counts()

,count
role,
React Developer,24
Workday Consultant,21
Peoplesoft Consultant,21
SQL Developer,14


Only PDF, DOCX, and DOC file formats are accepted; all other formats are removed from the dataset.

In [19]:
df["ext"].value_counts()

,count
ext,
docx,52
doc,26
txt,1
pdf,1


In [20]:
df[df["content"]=="Invalid Resume Content"]

,filename,content,ext,role
28,apple.txt,Invalid Resume Content,txt,Peoplesoft Consultant


In [21]:
df=df[df["content"]!="Invalid Resume Content"]

In [22]:
df[df["content"]=="Invalid Resume Content"]

,filename,content,ext,role


In [23]:
df["role"].value_counts()

,count
role,
React Developer,24
Workday Consultant,21
Peoplesoft Consultant,20
SQL Developer,14


In [24]:
df.to_csv("resume_cleandataset_labeled.csv", index=False)
print("CSV 'resume_cleandataset_labeled.csv' cleaned with invalid resume content.")

CSV 'resume_cleandataset_labeled.csv' cleaned with invalid resume content.
